In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").getOrCreate()

In [3]:
# Read the data into DataFrame
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ58W7SMO911M|          4|            2|          3|   N|                N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
| RATOTLA3OF70O|          5|            2|          2|   N|                N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|                N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|                N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|                N|
|R2XIM9LT335WHE|          4|            3|         10|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|     

In [5]:
# Print Scema
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [6]:
# Create dataframe where total_votes > 20
total_votes_filtered_df = vine_df.filter("total_votes>20")
total_votes_filtered_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
|R1AABFZH0J0C0C|          3|           17|         35|   N|                N|
|R3SJYLG07EHNE9|          2|           16|         34|   N|                N|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
|R1KPOIT5WT3NEF|          1|           13|         29|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
| R33IFUWCNYFSO|          2|            4|         45|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|     

In [7]:
# Create dataframe where helpful_votes / total_votes >= 50%
helpful_total_votes_df = total_votes_filtered_df.filter("(helpful_votes/total_votes)*100>=50")
helpful_total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|
|R2JSWNTAO83X3V|          5|          145|        184|   N|                N|
|R3THZWOBS1086J|          5|           46|         47|   N|                N|
|R160DHJPJ4B20R|          4|           28|         30|   N|                N|
|R27UDFNE6MY1LO|          2|           15|         24|   N|     

In [8]:
# Create new DataFrame with reviews as part of the Vine Program
paid_review_df = helpful_total_votes_df.filter("vine=='Y'")
paid_review_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [9]:
# Check the original dataframe for reviews where vine=='Y' to verify paid_review_df
vine_df.filter("vine=='Y'").show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2NKNL4WXWIWBG|          4|            3|          3|   Y|                Y|
| RAIG9ZQDMBF5P|          5|            3|          6|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+



In [10]:
# Create DataFrame with reviews that are not part of the Vine Program
unpaid_review_df = helpful_total_votes_df.filter("vine=='N'")
unpaid_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|
|R2JSWNTAO83X3V|          5|          145|        184|   N|                N|
|R3THZWOBS1086J|          5|           46|         47|   N|                N|
|R160DHJPJ4B20R|          4|           28|         30|   N|                N|
|R27UDFNE6MY1LO|          2|           15|         24|   N|     

In [11]:
# Determine the total number of reviews
helpful_total_votes_df.count()

378638

In [12]:
# Determine the number of 5-star reviews
five_star_df = helpful_total_votes_df.filter('star_rating==5')
five_star_df.count()

227734

In [13]:
# Determine the percentage of 5-star reviews for paid reviews
# Can't divide by zero; there are no paid reviews when filter total_votes > 20 

In [14]:
# Check percent of unpaid reviews in the five star reviews to confirm absence of paid reviews
five_star_df.filter("vine=='N'").count() / five_star_df.count() * 100

100.0

In [15]:
# Determine the percentage of 5-star reviews for unpaid reviews
unpaid_review_df.filter("star_rating==5").count() / unpaid_review_df.count() * 100

60.14557440087894